In [1]:
!pip install -q efficientnet_pytorch > /dev/null
!pip install -q torchcontrib
!!pip install --no-deps timm > /dev/null

[]

In [2]:
import sys
from glob import glob
import pandas as pd
from sklearn.model_selection import GroupKFold
import cv2
from skimage import io
import albumentations as A
import torch
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.nn import functional as F
from glob import glob
import sklearn
from torch import nn
import warnings
from torchvision import models
import torch.nn.functional as F
import types
import torch
from collections import OrderedDict
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from catalyst.data.sampler import BalanceClassSampler
import os
import timm
from datetime import datetime
from glob import glob
import time
import random
import sklearn
import random
import numpy as np
import scipy
from scipy import ndimage
from PIL import Image, ImageEnhance, ImageOps
from torchvision import transforms, utils
from torchcontrib.optim import SWA

warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=DeprecationWarning) 

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
/opt/conda/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
DATA_PATH = '../input/melanoma-merged-external-data-512x512-jpeg'

In [4]:
class Microscope:
    def __init__(self, p: float = 0.5):
        self.p = p

    def __call__(self, img):
        if random.random() < self.p:
            circle = cv2.circle((np.ones(img.shape) * 255).astype(np.uint8),
                        (img.shape[0]//2, img.shape[1]//2),
                        random.randint(img.shape[0]//2 - 3, img.shape[0]//2 + 15),
                        (0, 0, 0),
                        -1)

            mask = circle - 255
            img = np.multiply(img, mask)

        return img

    def __repr__(self):
        return f'{self.__class__.__name__}(p={self.p})'


In [5]:
def get_train_transforms():
    return A.Compose([
            A.RandomSizedCrop(min_max_height=(400, 400), height=768, width=768, p=0.5),
            A.RandomRotate90(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Resize(height=512, width=512, p=1),
            A.Cutout(num_holes=8, max_h_size=96, max_w_size=96, fill_value=0, p=0.5),
            ToTensorV2(p=1.0),                  
        ], p=1.0)

def get_valid_transforms():
    return A.Compose([
            A.Resize(height=768, width=768, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

# Metrics

In [6]:
from sklearn import metrics

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


class RocAucMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.y_true = np.array([0,1])
        self.y_pred = np.array([0.5,0.5])
        self.score = 0

    def update(self, y_true, y_pred):
        y_true = y_true.cpu().numpy().argmax(axis=1).clip(min=0, max=1).astype(int)
        # y_pred = 1 - nn.functional.softmax(y_pred, dim=1).data.cpu().numpy()[:,0]
        y_pred = nn.functional.softmax(y_pred, dim=1).data.cpu().numpy()[:,1]
        self.y_true = np.hstack((self.y_true, y_true))
        self.y_pred = np.hstack((self.y_pred, y_pred))
        self.score = sklearn.metrics.roc_auc_score(self.y_true, self.y_pred)

    @property
    def avg(self):
        return self.score

    
class APScoreMeter(RocAucMeter):
    def __init__(self):
        super(APScoreMeter, self).__init__()

    def update(self, y_true, y_pred):
        y_true = y_true.cpu().numpy().argmax(axis=1).clip(min=0, max=1).astype(int)
        y_pred = nn.functional.softmax(y_pred, dim=1).data.cpu().numpy()[:,1]
        self.y_true = np.hstack((self.y_true, y_true))
        self.y_pred = np.hstack((self.y_pred, y_pred))
        self.score = sklearn.metrics.average_precision_score(self.y_true, self.y_pred)

# Loss

In [7]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets)

        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss
        
class LabelSmoothing(nn.Module):
    def __init__(self, smoothing = 0.1):
        super(LabelSmoothing, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing

    def forward(self, x, target):
        if self.training:
            x = x.float()
            target = target.float()
            logprobs = torch.nn.functional.log_softmax(x, dim = -1)

            nll_loss = -logprobs * target
            nll_loss = nll_loss.sum(-1)
    
            smooth_loss = -logprobs.mean(dim=-1)

            loss = self.confidence * nll_loss + self.smoothing * smooth_loss

            return loss.mean()
        else:
            return torch.nn.functional.cross_entropy(x, target)
        

class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        return Dice_BCE        
    
class FocalDiceLoss(nn.Module):
    def __init__(self,alpha=1, gamma=2, logits=True, reduce=False,weight=None, size_average=True):
        super(FocalDiceLoss,self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce
   
    def forward(self, inputs, targets, smooth=1):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets)

        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        
        
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        if self.reduce:
            return torch.mean(F_loss)+dice_loss
        else:
            return F_loss+ dice_loss

# Net

In [8]:
def get_net():
    net = timm.create_model('efficientnet_b3',pretrained=True)
    net.classifier = nn.Linear(in_features=net.classifier.in_features, out_features=1000, bias=True)
#     print(net.classifier.in_features)
    return net

In [9]:
class MetaNet(nn.Module):
    def __init__(self):
        super(MetaNet, self).__init__()
        self.meta_before = FeedForwardNN()
        self.mainnet = get_net()
        self.metaaux = nn.Sequential(nn.BatchNorm1d(256),nn.ReLU(),nn.Linear(256,2))
        self.cnnaux = nn.Sequential(nn.BatchNorm1d(1000),nn.ReLU(),nn.Linear(1000,2)) 
        self.linear = nn.Linear(1256,2)
    def forward(self,inputs):
        x,meta_data = inputs
        meta_features = self.meta_before(meta_data)
        meta_out = self.metaaux(meta_features)
        cnn_features = self.mainnet(x)
        cnn_out = self.cnnaux(cnn_features)
        features = torch.cat((cnn_features,meta_features),dim=1)
        output = self.linear(features)
        return output,meta_out,cnn_out

In [10]:
class FeedForwardNN(nn.Module):

    def __init__(self):

        super().__init__()
        self.net1 = nn.Sequential(nn.Linear(3,64),nn.BatchNorm1d(64),nn.ReLU())
        self.net2 = nn.Sequential(nn.Linear(64,128),nn.BatchNorm1d(128),nn.ReLU())
        self.net3 = nn.Linear(128,256)
        
    def forward(self,inputs):
        output = self.net3(self.net2(self.net1(inputs)))
        return output

In [11]:
net = MetaNet()
# model_pth = '../input/b6fold2/3/last-checkpoint.bin'
# net.load_state_dict(torch.load(model_pth))

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b3_ra-a5e2fbc7.pth" to /root/.cache/torch/checkpoints/efficientnet_b3_ra-a5e2fbc7.pth


In [12]:
DATA_PATH = '../input/melanoma-merged-external-data-512x512-jpeg'
TRAIN_ROOT_PATH = '../input/siim-isic-melanoma-classification/jpeg/train'
TRAIN_ROOT_PATH0 = '../input/melanoma-merged-external-data-512x512-jpeg/512x512-dataset-melanoma/512x512-dataset-melanoma'

train = pd.read_csv(f'{DATA_PATH}/marking.csv')
test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
train['sex'] = train['sex'].fillna('na')
train['age_approx'] = train['age_approx'].fillna(0)
train['anatom_site_general_challenge'] = train['anatom_site_general_challenge'].fillna('na')
test['sex'] = test['sex'].fillna('na')
test['age_approx'] = test['age_approx'].fillna(0)
test['anatom_site_general_challenge'] = test['anatom_site_general_challenge'].fillna('na')

le = preprocessing.LabelEncoder()

train.sex = le.fit_transform(train.sex)
train.anatom_site_general_challenge = le.fit_transform(train.anatom_site_general_challenge)
train['age_approx'] = le.fit_transform(train['age_approx'])
test.sex = le.fit_transform(test.sex)
test.anatom_site_general_challenge = le.fit_transform(test.anatom_site_general_challenge)

scaler = StandardScaler()
x = scaler.fit_transform(train[['sex','age_approx','anatom_site_general_challenge']])
train.sex,train.age_approx,train.anatom_site_general_challenge = x[:,0],x[:,1],x[:,2]

In [13]:

def onehot(size, target):
    vec = torch.zeros(size, dtype=torch.float32)
    vec[target] = 1.
    return vec

class DatasetRetriever(Dataset):

    def __init__(self, df, image_ids, labels, transforms=None):
        super().__init__()
        self.image_ids = image_ids
        self.labels = labels
        self.transforms = transforms
        self.df = df

    def __getitem__(self, idx: int):
        image_id = self.image_ids[idx]
#         print(image_id)
        try:
            image = cv2.imread(f'{TRAIN_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
            image = image.astype(np.float32) / 255.0
        except:
            image = cv2.imread(f'{TRAIN_ROOT_PATH0}/{image_id}.jpg', cv2.IMREAD_COLOR)
            image = image.astype(np.float32) / 255.0
        record = self.df[self.df.image_id == image_id]
        meta_data = np.squeeze(record[['sex','age_approx','anatom_site_general_challenge']].values)
        label = self.labels[idx]
        meta_data = torch.tensor(meta_data,dtype=torch.float32)
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']
        target = onehot(2, label)
        return (image,meta_data), target

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def get_labels(self):
        return list(self.labels)

In [14]:
df_folds = pd.read_csv(f'{DATA_PATH}/folds.csv', index_col='image_id')

# Fitter

In [15]:
class Fitter:
    
    def __init__(self, model, device, config, folder):
        self.config = config
        self.epoch = 0

        self.base_dir = f'./{folder}'
        if not os.path.exists(self.base_dir):
            os.makedirs(self.base_dir)

        self.log_path = f'{self.base_dir}/log.txt'
        self.best_score = 0
        self.best_loss = 10**5
        self.best_ap = 0
        self.best_swa = 0
        
        self.model = model
        self.device = device

        
        self.base_optimizer = optim.AdamW(self.model.parameters(), lr=config.lr)
        self.optimizer = SWA(self.base_optimizer)
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        self.main_criterion = LabelSmoothing().to(self.device)
        self.cnn_criterion = FocalDiceLoss(logits=True).to(self.device)
        self.meta_criterion = FocalLoss(logits=True).to(self.device)
        self.log(f'Fitter prepared. Device is {self.device}')

    def fit(self, train_loader, validation_loader):
        for e in range(self.config.n_epochs):
            if self.config.verbose:
                lr = self.optimizer.param_groups[0]['lr']
                timestamp = datetime.utcnow().isoformat()
                self.log(f'\n{timestamp}\nLR: {lr}')

            t = time.time()

                    
            summary_main_loss,summary_meta_loss,summary_cnn_loss, roc_auc_scores, ap_scores = self.train_one_epoch(train_loader)
            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, main_loss: {summary_main_loss.avg:.5f},cnn_loss: {summary_cnn_loss.avg:.5f},meta_loss: {summary_meta_loss.avg:.5f}, roc_auc: {roc_auc_scores.avg:.5f}, ap: {ap_scores.avg:.5f}, time: {(time.time() - t):.5f}')
            
                
            t = time.time()
            
            if self.epoch >15 or roc_auc_scores.avg > 0.89:
                self.optimizer.update_swa()
                
            summary_main_loss,summary_meta_loss,summary_cnn_loss, roc_auc_scores, ap_scores = self.validation(validation_loader) 
            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, main_loss: {summary_main_loss.avg:.5f},cnn_loss: {summary_cnn_loss.avg:.5f},meta_loss: {summary_meta_loss.avg:.5f}, roc_auc: {roc_auc_scores.avg:.5f}, ap: {ap_scores.avg:.5f}, time: {(time.time() - t):.5f}')
            if summary_main_loss.avg < self.best_loss:
                self.best_loss = summary_main_loss.avg
                self.save_model(f'{self.base_dir}/best-loss-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-loss-checkpoint-*epoch.bin'))[:-1]:
                    os.remove(path)
                    
            if roc_auc_scores.avg > self.best_score:
                self.best_score = roc_auc_scores.avg
                self.save_model(f'{self.base_dir}/best-score-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-score-checkpoint-*epoch.bin'))[:-4]:
                    os.remove(path)
                    
            if ap_scores.avg > self.best_ap:
                self.best_ap = ap_scores.avg
                self.save_model(f'{self.base_dir}/best-ap-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-ap-checkpoint-*epoch.bin'))[:-1]:
                    os.remove(path)
            
            if self.config.validation_scheduler:
                self.scheduler.step(metrics=summary_main_loss.avg)
            self.epoch += 1
            if self.epoch == self.config.n_epochs:
                self.save_model(f'{self.base_dir}/last-checkpoint.bin')
                self.optimizer.swap_swa_sgd()
                print("SWA WEIGHTS")
                summary_main_loss,summary_meta_loss,summary_cnn_loss, roc_auc_scores, ap_scores = self.validation(validation_loader)
                print(roc_auc_scores)
                self.save_model(f'{self.base_dir}/last-checkpoint-swa.bin')
    def validation(self, val_loader):
        self.model.eval()
        
        summary_main_loss = AverageMeter()
        summary_meta_loss = AverageMeter()
        summary_cnn_loss = AverageMeter()
        
        roc_auc_scores = RocAucMeter()
        ap_scores = APScoreMeter()
        t = time.time()
        for step, (data, targets) in enumerate(val_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Val Step {step}/{len(val_loader)}, ' + \
                        f'main_loss: {summary_main_loss.avg:.5f},cnn_loss: {summary_cnn_loss.avg:.5f},meta_loss: {summary_meta_loss.avg:.5f}, roc_auc: {roc_auc_scores.avg:.5f}, ap: {ap_scores.avg:.5f} ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            with torch.no_grad():
                targets = targets.to(self.device).float()
                batch_size = data[0].shape[0]
                data[0] = data[0].to(self.device).float()
                data[1] = data[1].to(self.device).float()
                outputs = self.model(data)
                
                main_loss = self.main_criterion(outputs[0], targets)
                cnn_loss = self.cnn_criterion(outputs[2], targets)
                meta_loss = self.meta_criterion(outputs[1],targets)
                loss = main_loss + cnn_loss + meta_loss
                
                roc_auc_scores.update(targets, outputs[0])
                ap_scores.update(targets, outputs[0])
                summary_main_loss.update(main_loss.detach().item(), batch_size)
                summary_meta_loss.update(meta_loss.detach().item(), batch_size)
                summary_cnn_loss.update(cnn_loss.detach().item(), batch_size)

        return summary_main_loss,summary_meta_loss,summary_cnn_loss, roc_auc_scores, ap_scores

    def train_one_epoch(self, train_loader):
        self.model.train()
        summary_main_loss = AverageMeter()
        summary_meta_loss = AverageMeter()
        summary_cnn_loss = AverageMeter()
        roc_auc_scores = RocAucMeter()
        ap_scores = APScoreMeter()
        t = time.time()
        for step, (data, targets) in enumerate(train_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Train Step {step}/{len(train_loader)}, ' + \
                        f'main_loss: {summary_main_loss.avg:.5f},cnn_loss: {summary_cnn_loss.avg:.5f},meta_loss: {summary_meta_loss.avg:.5f}, roc_auc: {roc_auc_scores.avg:.5f}, ap: {ap_scores.avg:.5f} ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            targets = targets.to(self.device).float()
            data[0] = data[0].to(self.device).float()
            data[1] = data[1].to(self.device).float()
            batch_size = data[0].shape[0]

            self.optimizer.zero_grad()
            outputs = self.model(data)
            
            main_loss = self.main_criterion(outputs[0], targets)
            cnn_loss = self.cnn_criterion(outputs[2], targets)
            meta_loss = self.meta_criterion(outputs[1],targets)
            loss = main_loss + cnn_loss + meta_loss
            loss.backward()
            
            roc_auc_scores.update(targets, outputs[0])
            ap_scores.update(targets, outputs[0])
            
            summary_main_loss.update(main_loss.detach().item(), batch_size)
            summary_meta_loss.update(meta_loss.detach().item(), batch_size)
            summary_cnn_loss.update(cnn_loss.detach().item(), batch_size)

            self.optimizer.step()
            
            if self.config.step_scheduler:
                self.scheduler.step()

        return summary_main_loss,summary_meta_loss,summary_cnn_loss, roc_auc_scores, ap_scores
    
    def save_model(self, path):
        self.model.eval()
        torch.save(self.model.state_dict(),path)

    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_score': self.best_score,
            'best_ap': self.best_ap,
            'best_loss': self.best_loss,
            'epoch': self.epoch,
        }, path)

    def load(self, path):
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.best_score = checkpoint['best_score']
        self.best_ap = checkpoint['best_ap']
        self.best_loss = checkpoint['best_loss']
        self.epoch = checkpoint['epoch']
        
    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

In [16]:
class TrainGlobalConfig:
    num_workers = 2
    batch_size = 8
    n_epochs = 20
    lr = 0.00005

    # -------------------
    verbose = True
    verbose_step = 1
    # -------------------

    # --------------------
    step_scheduler = False  # do scheduler.step after optimizer.step
    validation_scheduler = True  # do scheduler.step after validation stage loss

    
    SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
    scheduler_params = dict(
        mode='min',
        factor=0.8,
        patience=1,
        verbose=False, 
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0, 
        min_lr=1e-8,
        eps=1e-08
    )
    # --------------------

# Save all states for "honest" training of folds

In [17]:
fitter = Fitter(model=net, device=torch.device('cuda:0'), config=TrainGlobalConfig, folder='base_state')
BASE_STATE_PATH = f'{fitter.base_dir}/base_state.bin'
fitter.save(BASE_STATE_PATH)

Fitter prepared. Device is cuda:0


In [18]:
def train_fold(fold_number):

    train_dataset = DatasetRetriever(df = train,
        image_ids=df_folds[df_folds['fold'] != fold_number].index.values,
        labels=df_folds[df_folds['fold'] != fold_number].target.values,
        transforms=get_train_transforms(),
    )

    df_val = df_folds[(df_folds['fold'] == fold_number) & (df_folds['source'] == 'ISIC20')]

    validation_dataset = DatasetRetriever(df= train,
        image_ids=df_val.index.values,
        labels=df_val.target.values,
        transforms=get_valid_transforms(),
    )

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        sampler=BalanceClassSampler(labels=train_dataset.get_labels(), mode="downsampling"),
        batch_size=TrainGlobalConfig.batch_size,
        pin_memory=False,
        drop_last=True,
        num_workers=TrainGlobalConfig.num_workers,
    )
    val_loader = torch.utils.data.DataLoader(
        validation_dataset, 
        batch_size=TrainGlobalConfig.batch_size,
        num_workers=TrainGlobalConfig.num_workers,
        shuffle=False,
        sampler=SequentialSampler(validation_dataset),
        pin_memory=False,
    )

    fitter = Fitter(model=net.cuda(), device=torch.device('cuda:0'), config=TrainGlobalConfig, folder=f'{fold_number}')
    fitter.load(BASE_STATE_PATH)
    print("strat")
    fitter.fit(train_loader, val_loader)

In [ ]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning) 

train_fold(fold_number=0)

### Inference(Without TTA)
#### For TTA just replace get_valid_transforms() with get_train_transforms() and predict the output multiple times


In [20]:
model_pth = '../input/b6fold2/4/best-ap-checkpoint-010epoch.bin'  ##path to be changed multiple times
net.load_state_dict(torch.load(model_pth))
net = net.cuda()

test_dataset = DatasetRetriever(df= test,
        image_ids=test.image_name.values,
        labels=np.zeros(len(test), dtype = int),
        transforms=get_valid_transforms(),
    )

train_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=TrainGlobalConfig.batch_size,
        pin_memory=False,
        num_workers=8,
        shuffle = False
    )

from torch.nn.functional import softmax

from tqdm import tqdm
result = []
with torch.no_grad():
    for i, (data, targets) in tqdm(enumerate(train_loader), total = len(train_loader)):
        torch.cuda.empty_cache()
        data[0] = data[0].cuda().float()
        data[1] = data[1].cuda()
        output,meta_out,cnn_out = net(data)
        out = 1/3*(softmax(output, dim = 1) + softmax(meta_out, dim = 1)  + softmax(cnn_out, dim = 1) )
        result.extend(out[:, 1].detach().cpu().tolist())
        
submission = pd.DataFrame({'image_name' : test.image_name, 'target' : result})
submission.to_csv('tfb6_fold4_ap.csv', header = submission.columns, index =  False )

In [21]:
# len(result)